In [35]:
import time
import numpy as np
import pandas as pd
import os

from hydesign.assembly.hpp_assembly import hpp_model
from hydesign.examples import examples_filepath

In [36]:
sites = pd.read_csv('overview.csv', index_col=0)
example = 0

ex_site = sites.iloc[int(example),:]

print('Selected example site:')
print('---------------------------------------------------')
print(ex_site.T)

longitude = ex_site['longitude']
latitude = ex_site['latitude']
altitude = ex_site['altitude']
input_ts_fn = os.path.join(os.getcwd(), ex_site['input_ts_fn'])
sim_pars_fn = os.path.join(os.getcwd(), ex_site['sim_pars_fn'])

hpp = hpp_model(
        latitude,
        longitude,
        altitude,
        num_batteries = 10,
        work_dir = './',
        sim_pars_fn = sim_pars_fn,
        input_ts_fn = input_ts_fn,
        save_battry_rf_filename='battery_rf.csv',
)

inputs = dict(
clearance =	60,
sp =	287.6391934,
p_rated =	10,
Nwt	=	25,
wind_MW_per_km2 =	4.782274215,
solar_MW =	223.2191223,
surface_tilt =	50,
surface_azimuth = 210,
DC_AC_ratio	=1.295598428,
b_P =75.27402422,
b_E_h=	4.451612903,
cost_of_battery_P_fluct_in_peak_price_ratio	=0.580645161,)

Selected example site:
---------------------------------------------------
case                                              Europe
name                                    France_good_wind
longitude                                      -0.864258
latitude                                       48.744116
altitude                                             302
input_ts_fn      hyd_input/input_ts_France_good_wind.csv
sim_pars_fn                       hyd_input/hpp_pars.yml
price_fn                     hyd_input/2030-EL_PRICE.csv
price_col                                           FR_R
H2_demand_col                    hyd_input/H2_demand.csv
Name: 0, dtype: object

Fixed parameters on the site
-------------------------------
longitude = -0.864258
latitude = 48.744116
altitude = 302

input_ts_fn length is not a complete number of years (hyDesign handles years as 365 days).
The file has been modified and stored in ./input_ts_modified.csv


In [37]:
start = time.time()
outs = hpp.evaluate(**inputs)
hpp.print_design([v for k,v in inputs.items()], outs)
end = time.time()
print('exec. time [min]:', (end - start)/60 )
print(hpp.prob['NPV_over_CAPEX'])


Design:
---------------
clearance [m]: 60.000
sp [W/m2]: 287.639
p_rated [MW]: 10.000
Nwt: 25.000
wind_MW_per_km2 [MW/km2]: 4.782
solar_MW [MW]: 223.219
surface_tilt [deg]: 50.000
surface_azimuth [deg]: 210.000
DC_AC_ratio: 1.296
b_P [MW]: 75.274
b_E_h [h]: 4.452
cost_of_battery_P_fluct_in_peak_price_ratio: 0.581


NPV_over_CAPEX: 0.134
NPV [MEuro]: 57.419
IRR: 0.072
LCOE [Euro/MWh]: 33.631
Revenues [MEuro]: 48.109
CAPEX [MEuro]: 429.684
OPEX [MEuro]: 6.900
Wind CAPEX [MEuro]: 275.586
Wind OPEX [MEuro]: 5.599
PV CAPEX [MEuro]: 65.197
PV OPEX [MEuro]: 1.301
Batt CAPEX [MEuro]: 22.236
Batt OPEX [MEuro]: 0.000
Shared CAPEX [MEuro]: 66.665
Shared Opex [MEuro]: 0.000
penalty lifetime [MEuro]: 0.000
AEP [GWh]: 1204.470
GUF: 0.458
grid [MW]: 300.000
wind [MW]: 250.000
solar [MW]: 223.219
Battery Energy [MWh]: 335.091
Battery Power [MW]: 75.274
Total curtailment [GWh]: 124.715
Awpp [km2]: 52.276
Apvp [km2]: 2.737
Plant area [km2]: 52.276
Rotor diam [m]: 210.393
Hub height [m]: 165.196
Number 

In [38]:
from hopp.simulation import HoppInterface

In [31]:
hi = HoppInterface(os.path.join(os.getcwd(), "hopp_input/08-wind-solar-france.yaml"))

FLORIS is the system model...


In [32]:
hi.simulate(25)

Simulating wind farm output in FLORIS...
Wind annual energy:  663116990.2109435


In [33]:
hybrid_plant = hi.system

annual_energies = hybrid_plant.annual_energies
npvs = hybrid_plant.net_present_values
cf = hybrid_plant.capacity_factors

wind_installed_cost = hybrid_plant.wind.total_installed_cost
solar_installed_cost = hybrid_plant.pv.total_installed_cost
battery_installed_cost = hybrid_plant.battery.total_installed_cost
hybrid_installed_cost = hybrid_plant.grid.total_installed_cost

print("Wind Installed Cost [M$]: {}".format(wind_installed_cost/1e6))
print("Solar Installed Cost [M$]: {}".format(solar_installed_cost/1e6))
print("Battery Installed Cost [M$]: {}".format(battery_installed_cost/1e6))
print("Hybrid Installed Cost [M$]: {}\n".format(hybrid_installed_cost/1e6))

print("Wind NPV [M$]: {}".format(hybrid_plant.net_present_values.wind/1e6))
print("Solar NPV [M$]: {}".format(hybrid_plant.net_present_values.pv/1e6))
print("Hybrid NPV [M$]: {}\n".format(hybrid_plant.net_present_values.hybrid/1e6))

print("Annual Energies [kW]",annual_energies)
print("Capacity Factors",cf)
print("NPV [$]", npvs)

print("LCOE [cents/kWh]", hybrid_plant.lcoe_real,"\n")

print("Hybrid LCOE [$/MWh]", hybrid_plant.lcoe_real['hybrid']*10)

Wind Installed Cost [M$]: 241.227
Solar Installed Cost [M$]: 54.989034800000006
Battery Installed Cost [M$]: 17.5693665
Hybrid Installed Cost [M$]: 313.7854013

Wind NPV [M$]: 43.13380019290942
Solar NPV [M$]: 2.5456121654541493
Hybrid NPV [M$]: 39.73664302291751

Annual Energies [kW] {"pv": 277789162.6113242, "wind": 663116990.2109435, "battery": -2859748.762284336, "hybrid": 936837216.2761368}
Capacity Factors {"pv": 18.48628659595806, "wind": 30.279314621504266, "battery": 0, "hybrid": 21.56588686123423}
NPV [$] {"pv": 2545612.1654541492, "wind": 43133800.19290942, "battery": -2140217.4264282733, "hybrid": 39736643.02291751}
LCOE [cents/kWh] {"pv": 2.4773060421462083, "wind": 2.9739200415014664, "battery": 5.048881089149761, "hybrid": 3.100584251886565} 

Hybrid LCOE [$/MWh] 31.00584251886565


In [34]:
import json

with open("singleowner.json", 'w') as f:
    dat = hybrid_plant.grid._financial_model.export()
    d = dict()
    for k, v in dat.items():
        d.update(v)
    json.dump(d, f)

In [61]:
euro_per_dollar = 1.0
df = pd.DataFrame({'HOPP': [wind_installed_cost/1e6*euro_per_dollar,
                            solar_installed_cost/1e6*euro_per_dollar,
                            battery_installed_cost/1e6*euro_per_dollar,
                            hybrid_installed_cost/1e6*euro_per_dollar,
                            hybrid_plant.net_present_values.hybrid/1e6*euro_per_dollar,
                           ],
                   'HyDesign': [float(hpp.prob['wpp_cost.CAPEX_w']/1e6),
                                float(hpp.prob['pvp_cost.CAPEX_s']/1e6),
                                float(hpp.prob['battery_cost.CAPEX_b']/1e6),
                                float(hpp.prob['CAPEX']/1e6),
                                float(hpp.prob['NPV']/1e6),
                                ] },
                  index=['CAPEX Wind [M€]',
                         'CAPEX Solar [M€]',
                         'CAPEX Battery [M€]',
                         'CAPEX Hybrid [M€]',
                         'NPV Hybrid [M€]',
                        ])

In [62]:
df

,HOPP,HyDesign
CAPEX Wind [M€],241.227000,275.586490
CAPEX Solar [M€],54.989035,65.196875
CAPEX Battery [M€],17.569367,22.235916
CAPEX Hybrid [M€],313.785401,429.684196
NPV Hybrid [M€],39.736643,57.418568


In [41]:
hpp.prob.model.list_outputs()

45 Explicit Output(s) in 'model'

varname                               val                   prom_name                                   
------------------------------------  --------------------  --------------------------------------------
abl
  wst                                 |817.46032848|        abl.wst                                     
genericWT
  ws                                  |117.70938833|        genericWT.ws                                
  pc                                  |5.71613057|          genericWT.pc                                
  ct                                  |3.31364971|          genericWT.ct                                
genericWake
  pcw                                 |1413.01276995|       genericWake.pcw                             
wpp
  wind_t                              |12206.75107181|      wpp.wind_t                                  
pvp
  solar_t                             |6226.64451123|       pvp.solar_t                     

[('abl.wst',
  {'val': array([12.5730722 , 11.58387731, 11.8185967 , ..., 12.50101548,
          13.00332079, 13.29007885]),
   'prom_name': 'abl.wst'}),
 ('battery_cost.CAPEX_b',
  {'val': array([22235916.21067887]), 'prom_name': 'battery_cost.CAPEX_b'}),
 ('battery_cost.OPEX_b',
  {'val': array([0.]), 'prom_name': 'battery_cost.OPEX_b'}),
 ('battery_degradation.SoH',
  {'val': array([0.99995228, 0.9999375 , 0.99992271, ..., 0.71693253, 0.71692954,
          0.71692954]),
   'prom_name': 'battery_degradation.SoH'}),
 ('battery_degradation.n_batteries',
  {'val': array([2.]), 'prom_name': 'battery_degradation.n_batteries'}),
 ('battery_loss_in_capacity_due_to_temp.SoH_all',
  {'val': array([0.97811107, 0.97733277, 0.97680722, ..., 0.68140446, 0.68106479,
          0.68097746]),
   'prom_name': 'battery_loss_in_capacity_due_to_temp.SoH_all'}),
 ('ems.wind_t_ext',
  {'val': array([216.63691369, 211.60142827, 213.3433593 , ..., 216.51662194,
          217.35208738, 217.56441215]),
   'pro